# Example: Question Answering Task
An example describing how to integrate eyeball-plus-plus for a simple task which answers questions based on a given context

In [ ]:
%pip install eyeball_pp openai pyyaml rich

### 1. Define your LLM task
Define the method and add eyeball decorator to record

In [2]:
import eyeball_pp
import openai

openai.api_key = "your-openai-api-key"

# Setting a sample_rate of 1 means that every call to the ask function will be recorded.
# You might want to change this on production to a lower value like 0.1 if you only want to record 10% of the calls.
eyeball_pp.set_config(sample_rate=1)

@eyeball_pp.record_task(args_to_record=["context", "question"])
def ask(context: str, question: str) -> str:
    # You can write arbitrary code here, the only thing the eval framework
    # cares about is the input and output of this function.
    # In this case the inputs context and question are recorded and the output which is the return value of this function is recorded.

    system = """
    You are trying to answer a question strictly using the information provided in the context. Reply I don't know if you don't know the answer.
    """

    prompt = f"""
    Context: {context}
    Question: {question}
    """

    # eval params can be set when you are trying to evaluate this agent
    # with different parameters eg. different models, providers or hyperparameters like temperature
    model = eyeball_pp.get_eval_param("model") or "gpt-3.5-turbo"
    temperature = eyeball_pp.get_eval_param("temperature") or 0.5

    # Note you can use any arbitrary LLM here, this example uses the openai API but you can 
    # use anthropic claude, or any other open source LLM
    output = openai.ChatCompletion.create(  # type: ignore
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
    )["choices"][0]["message"][
        "content"
    ]  # type: ignore
    return output

### 2. Initial Run
Run the task with a few different inputs

In [6]:
answer1 = ask(context="The quick brown fox jumps over the lazy dog",
    question="What color is the fox?",
)
print(answer1)

answer2 = ask(context="The lazy dog which is not brown jumps over the quick brown fox",
    question="What color is the dog?",
)
print(answer2)

I don't know.
I don't know.


### 3. Re-run with new Parameters
Rerun recorded examples with different eval params

In [4]:
for input_vars in eyeball_pp.rerun_recorded_examples(
    {"model": "gpt-3.5-turbo-16k", "temperature": 0}
):
    answer = ask(input_vars["context"], input_vars["question"])
    print(f'answer="{answer}"')


Will rerun 2 inputs for task:`ask`

Rerunning input #0:
context="The quick brown fox jumps over the lazy dog"
question="What color is the fox?"

Using eval params: {'model': 'gpt-3.5-turbo-16k', 'temperature': 0}
answer="I don't know."

Rerunning input #1:
context="The lazy dog which is not brown jumps over the quick brown fox"
question="What color is the dog?"

Using eval params: {'model': 'gpt-3.5-turbo-16k', 'temperature': 0}
answer="I don't know."


### 4. Compare Results
Compare the recorded checkpoints across different runs using an objective

In [5]:
eyeball_pp.compare_recorded_checkpoints(
    task_objective="This agent tries to answer questions given a context. Verify that the agent answers the question correctly and that the answer is only based on the context."
)

Comparing 2 inputs for task:`ask`

Input #0 - Running 1 comparison(s)
[improvement] task output improved from checkpoint 2023-08-01T19:24:26.587752  to 2023-08-01T19:24:32.630857 (temperature=0, model=gpt-3.5-turbo-16k)

Input #1 - Running 1 comparison(s)
[improvement] task output improved from checkpoint 2023-08-01T19:24:27.625472  to 2023-08-01T19:24:33.310973 (temperature=0, model=gpt-3.5-turbo-16k)

Summary:
---------
Your most sucessful re-runs:
2023-08-01T19:24:32.630821: 2/2 successes

Your most sucessful params:
model=gpt-3.5-turbo-16k,temperature=0: 2/2 successes


                                                Comparison Results                                                 
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ context              ┃ question             ┃ latest_checkpoint    ┃ previous_checkpoint ┃ the_checkpoint_befo… ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ "The quick brown fox │ "What color is the   │ 📈 +ve               │                     │                      │
│ jumps over the lazy  │ fox?"                │ (model=gpt-3.5-turb… │                     │                      │
│ dog"                 │                      │ temperature=0)       │                     │                      │
│ "The lazy dog which  │ "What color is the   │ 📈 +ve               │                     │                      │
│ is not brown jumps   │ dog?"                │ (model=gpt-3.5-turb… │                     │                      │
│ over the quick brown │                      │ temperature=0)       │                     │                      │
│ fox"                 │                      │                      │                     │                      │
└──────────────────────┴──────────────────────┴──────────────────────┴─────────────────────┴──────────────────────┘